In [ ]:
#| default_exp learning_context

# Learning Context

> Core module for managing learning context (memory -> LOs, prior cases, knowledge gaps, feedback preferences)

## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any, Tuple
import logging
from datetime import datetime
import json
from pathlib import Path
import gradio as gr

C:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
def setup_logger(name: str) -> logging.Logger:
    """Set up module logger"""
    logger = logging.getLogger(name)
    if not logger.handlers:
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
        )
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
    return logger

logger = setup_logger(__name__)

## Learning Context Management

> Core module for managing student learning context and history

The system needs to track and handle student information for personalised education.  

This module handles:
* Tracking learning objectives
* Managing case history
* Monitoring knowledge gaps
* Customizing feedback preferences

and finally:
* Context persistence


In [ ]:
#| export

class LearningContext:
    """
    Manages the dynamic learning context for each student.
    
    Attributes:
        learning_objectives (Dict[str, Any]): Current learning goals
        recent_cases (List[Dict]): History of recent case interactions
        identified_gaps (Dict[str, float]): Topics and confidence scores
        feedback_preferences (Dict[str, float]): Weighted feedback preferences
    """
    
    def __init__(self, context_file: Optional[Path] = None):
        """
        Initialize learning context, optionally loading from file.
        
        Args:
            context_file: Optional path to saved context
        """
        self.learning_objectives: Dict[str, Any] = {}
        self.recent_cases: List[Dict] = []
        self.identified_gaps: Dict[str, float] = {}
        self.feedback_preferences: Dict[str, float] = {
            "clinical_reasoning": 1.0,
            "medical_knowledge": 1.0,
            "presentation_skills": 1.0,
            "differential_building": 1.0
        }
        
        if context_file and context_file.exists():
            self.load_context(context_file)
            
        logger.info("Learning context initialized")
        
    def update_from_interaction(self, interaction_data: Dict):
        """
        Update context based on new learning interaction.
        
        Args:
            interaction_data: Dict containing interaction details
        """
        # Update recent cases
        if 'case' in interaction_data:
            self.recent_cases.append({
                'timestamp': datetime.now().isoformat(),
                'content': interaction_data['case'],
                'feedback': interaction_data.get('feedback', ''),
                'learning_points': interaction_data.get('learning_points', [])
            })
            # Keep only last 5 cases
            self.recent_cases = self.recent_cases[-5:]
        
        # Update knowledge gaps
        if 'gaps' in interaction_data:
            self.identified_gaps.update(interaction_data['gaps'])
            # Remove gaps that have been addressed
            self.identified_gaps = {
                topic: score for topic, score in self.identified_gaps.items()
                if score < 0.8  # Only keep significant gaps
            }
            
    def get_context_prompt(self) -> str:
        """
        Generate a context-aware prompt for the tutor.
        
        Returns:
            str: Formatted context prompt
        """
        context_parts = []
        
        # Add learning objectives
        if self.learning_objectives:
            context_parts.append("Current Learning Objectives:")
            for obj, details in self.learning_objectives.items():
                context_parts.append(f"- {obj}: {details}")
        
        # Add knowledge gaps
        if self.identified_gaps:
            context_parts.append("\nAreas Needing Focus:")
            for topic, confidence in self.identified_gaps.items():
                if confidence < 0.7:
                    context_parts.append(f"- {topic} (confidence: {confidence:.2f})")
        
        # Add recent case context
        if self.recent_cases:
            context_parts.append("\nRecent Case Context:")
            for case in self.recent_cases[-2:]:
                summary = case['content'][:100] + "..."
                points = "\n  ".join(case.get('learning_points', []))
                context_parts.append(f"- Previous case: {summary}")
                if points:
                    context_parts.append(f"  Key points:\n  {points}")
        
        # Add feedback preferences
        context_parts.append("\nFeedback Focus:")
        for aspect, weight in self.feedback_preferences.items():
            if weight > 1.0:
                context_parts.append(
                    f"- Emphasize {aspect.replace('_', ' ')} (weight: {weight:.1f})"
                )
        
        return "\n".join(context_parts)

    def save_context(self, file_path: Path):
        """
        Save context to file.
        
        Args:
            file_path: Path to save context
        """
        context_data = {
            'learning_objectives': self.learning_objectives,
            'recent_cases': self.recent_cases,
            'identified_gaps': self.identified_gaps,
            'feedback_preferences': self.feedback_preferences
        }
        
        with open(file_path, 'w') as f:
            json.dump(context_data, f, indent=2)
    
    def load_context(self, file_path: Path):
        """
        Load context from file.
        
        Args:
            file_path: Path to load context from
        """
        try:
            with open(file_path, 'r') as f:
                context_data = json.load(f)
            
            self.learning_objectives = context_data.get('learning_objectives', {})
            self.recent_cases = context_data.get('recent_cases', [])
            self.identified_gaps = context_data.get('identified_gaps', {})
            self.feedback_preferences = context_data.get('feedback_preferences', {})
        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")

In [ ]:
#| export

class ClinicalTutor:
    """Adaptive clinical education module"""
    
    def __init__(self):
        self.api_key: str = os.getenv("OPENROUTER_API_KEY")
        if not self.api_key:
            raise ValueError("OpenRouter API key not found in environment variables")
            
        self.api_url: str = "https://openrouter.ai/api/v1/chat/completions"
        self.model: str = "anthropic/claude-3-sonnet"
        
        # Initialize learning context
        self.learning_context = LearningContext()
    
    def _get_completion(self, messages: List[Dict], temperature: float = 0.7) -> str:
        """Get completion from OpenRouter API"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature
        }
        
        try:
            response = requests.post(
                self.api_url,
                headers=headers,
                json=data
            )
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
        except Exception as e:
            logger.error(f"Error in API call: {str(e)}")
            raise

    def _build_system_prompt(self) -> str:
        """Build dynamic system prompt based on context"""
        base_prompt = """You are an experienced medical educator providing adaptive feedback 
        and guidance. Consider the student's current learning context, recent cases, and 
        identified areas for improvement when providing feedback."""
        
        context = self.learning_context.get_context_prompt()
        if context:
            base_prompt += f"\n\nStudent Context:\n{context}"
            
        return base_prompt

    async def process_interaction(
        self, 
        content: str,
        interaction_type: str = "case_presentation"
    ) -> Dict[str, Any]:
        """
        Process any type of learning interaction with context-aware feedback
        
        Args:
            content: The student's input (case presentation, question, etc.)
            interaction_type: Type of interaction (defaults to case_presentation)
            
        Returns:
            dict: Processed response with feedback and identified learning points
        """
        try:
            system_prompt = self._build_system_prompt()
            
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"""Process this {interaction_type}:
                
                {content}
                
                Provide feedback addressing:
                1. Key learning points
                2. Areas for improvement
                3. Relevant connections to recent cases (if any)
                4. Suggested focus areas for future learning
                
                Return your response in JSON format with these keys:
                - feedback: Main feedback message
                - learning_points: List of key learning points
                - gaps: Dict of identified knowledge gaps
                - suggestions: List of specific improvement suggestions"""}
            ]
            
            response = self._get_completion(messages)
            processed_response = json.loads(response)
            
            # Update learning context
            self.learning_context.update_from_interaction({
                'case': content,
                'feedback': processed_response['feedback'],
                'gaps': processed_response['gaps']
            })
            
            return processed_response
            
        except Exception as e:
            logger.error(f"Error processing interaction: {str(e)}")
            return {
                "feedback": "An error occurred processing your input.",
                "learning_points": [],
                "gaps": {},
                "suggestions": ["Please try again or contact support."]
            }

    def update_learning_objectives(self, objectives: Dict[str, Any]):
        """Update learning objectives in context"""
        self.learning_context.learning_objectives.update(objectives)
        
    def update_feedback_preferences(self, preferences: Dict[str, float]):
        """Update feedback preferences in context"""
        self.learning_context.feedback_preferences.update(preferences)


## Tests

In [ ]:
def test_learning_context():
    """Test LearningContext functionality"""
    context = LearningContext()
    
    # Test interaction updates
    interaction_data = {
        'case': 'Test case presentation',
        'feedback': 'Good structure',
        'learning_points': ['Point 1', 'Point 2'],
        'gaps': {'topic1': 0.6, 'topic2': 0.8}
    }
    
    context.update_from_interaction(interaction_data)
    
    assert len(context.recent_cases) == 1
    assert 'topic1' in context.identified_gaps
    assert context.identified_gaps['topic1'] == 0.6
    
    # Test persistence
    temp_path = Path("test_context.json")
    context.save_context(temp_path)
    
    new_context = LearningContext(temp_path)
    assert new_context.identified_gaps == context.identified_gaps
    
    # Cleanup
    temp_path.unlink()
    
    print("All learning context tests passed!")

# Run tests
if __name__ == "__main__":
    test_learning_context()

2025-01-18 23:31:55,450 - __main__ - INFO - Learning context initialized
2025-01-18 23:31:55,457 - __main__ - INFO - Learning context initialized


All learning context tests passed!


In [ ]:
#| export

class LearningInterface:
    """Gradio interface for clinical learning interactions"""
    
    def __init__(self):
        self.tutor = ClinicalTutor()
        
    def process_input(
        self, 
        message: str, 
        history: List[Tuple[str, str]]
    ) -> Tuple[List[Tuple[str, str]], str]:
        """Process user input and generate context-aware response"""
        try:
            if not message.strip():
                return history, "Please provide your case or question."
            
            response = self.tutor.process_interaction(message)
            
            # Format response for display
            formatted_response = f"""Feedback: {response['feedback']}

Key Learning Points:
{chr(10).join(f'• {point}' for point in response['learning_points'])}

Suggestions:
{chr(10).join(f'• {sugg}' for sugg in response['suggestions'])}"""
            
            history.append((message, formatted_response))
            return history, ""
            
        except Exception as e:
            logger.error(f"Error in interface: {str(e)}")
            return history, "An error occurred. Please try again."
    
    def create_interface(self) -> gr.Blocks:
        """Create and configure the Gradio interface"""
        with gr.Blocks(title="Clinical Learning Assistant") as interface:
            gr.Markdown("# Clinical Learning Assistant")
            
            with gr.Tab("Case Discussion"):
                chatbot = gr.Chatbot(height=400)
                msg = gr.Textbox(
                    label="Present your case or ask a question",
                    placeholder="Start with the chief complaint...",
                    lines=3
                )
                clear = gr.Button("Clear")
            
            with gr.Tab("Learning Preferences"):
                # Add controls for updating learning objectives and feedback preferences
                pass
                
            msg.submit(
                self.process_input,
                inputs=[msg, chatbot],
                outputs=[chatbot, msg]
            )
            clear.click(lambda: ([], ""), outputs=[chatbot, msg])
            
        return interface


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()